Celem ćwiczenia jest implementacja algorytmu min-max z przycinaniem α−β.
Dla różnych ruchów o tej samej jakości, algorytm powinien zwracać losowy z
nich.
Następnie należy wykorzystać implementację do porównania jakości dla różnych
głębokości przeszukiwania dla gry kółko i krzyżyk na planszy NxN. W
raporcie powinny być umieszczone wyniki turnieju w grę kółko i krzyżyk, w
którym biorą udział gracze sterowani algorytmem z różnymi głębokościami przeszukiwania.

# Ćwiczenie 3 - Dwuosobowe gry deterministyczne
## Implementacja algorytmu min-max z przycinaniem α−β

### Importowanie modułów
- potrzebnych do wykonania ćwiczenia. W realizacji algorytmu korzystam z biblioteki math, random, numpy oraz copy,

In [2]:
import numpy as np
from enum import Enum
from typing import List
from copy import deepcopy

### Importowanie modułów
- potrzebnych do wykonania ćwiczenia. W realizacji algorytmu korzystam z biblioteki math, random, numpy oraz copy,

### Zdefiniowanie stanów planszy:
- za pomocą użycia Enum

In [3]:
class BoardStatus(Enum):
    EMPTY = 0
    X = 1
    O = 2

### Klasa TicTacToe
- badanej przez nasz algorytm gry

In [1]:
class TicTacToe:  # zmienić nazwę na gameboard
    def __init__(self, size_row: int):
        self.size_row = size_row
        self.game_board = np.zeros((size_row, size_row), dtype=int)
        self.heuristic_matrix = self.create_heuristic_matrix()
        self.heuristic_score = self.heuristic_function()
        self.children = 0

    def set_heuristic_score(self, new_heuristic_score: float):
        self.heuristic_score = new_heuristic_score

    def set_children(self, new_children: List['TicTacToe']):
        self.children = sorted(new_children, key=self.heuristic_score(), reverse=True)

    def create_heuristic_matrix(self):
        heuristic_matrix = np.zeros((self.size_row, self.size_row), dtype=int)

        for each_row in range(self.size_row):
            for each_value in range(self.size_row):
                heuristic_matrix[each_row][each_value] = 2

        for position in range(self.size_row):
            heuristic_matrix[position][position] = 3

        for position in range(self.size_row):
            heuristic_matrix[position][self.size_row - 1 - position] = 3

        if self.size_row % 2 == 1:
            heuristic_matrix[self.size_row // 2][self.size_row // 2] = 4

        return heuristic_matrix

    def heuristic_function(self):
        configuration_score = 0
        for each_row in range(self.size_row):
            for each_value in range(self.size_row):
                if self.game_board[each_row][each_value] == BoardStatus.O.value:
                    configuration_score += self.heuristic_matrix[each_row][each_value]
                if self.game_board[each_row][each_value] == BoardStatus.X.value:
                    configuration_score -= self.heuristic_matrix[each_row][each_value]
        return configuration_score


    def evaluate_children(self, is_maximizing_player: bool) -> List:
        children = []
        for each_row in range(self.size_row):
            for each_possible_children in range(self.size_row):
                if self.is_move_possible((each_row, each_possible_children)):
                    child = deepcopy(self)
                    child.make_move((each_row, each_possible_children), is_maximizing_player)
                    child.heuristic_function()
                    children.append(child)
        return children

    def evaluate_win(self, who_won):
        if who_won == BoardStatus.O.value:
            self.set_heuristic_score(float('inf'))
        else:
            self.set_heuristic_score(float('-inf'))

    def is_terminal(self):
        possible_wins = [self.horizontal_win(), self.vertical_win(), self.first_diagonal_win(), self.second_diagonal_win()]
        for each_possible_win in possible_wins:
            if each_possible_win is not None and each_possible_win != 0:
                self.evaluate_win(each_possible_win)  # nie jestem pewna jeszcze czy ma się to tutaj znajdować...
                return True, each_possible_win
        # sprawdzanie, czy ktoś wygrywa
        for each_row in range(self.size_row):
            for each_column in range(self.size_row):
                if self.game_board[each_row][each_column] == 0:
                    return False, None
            return True, 0

    def is_move_possible(self, provided_node):
        if provided_node[0] in range(self.size_row) and provided_node[1] in range(self.size_row):  # nie wiem, czy ten warunek jest potrzebny
            if self.game_board[provided_node] == 0:
                return True
        return False


    def horizontal_win(self):  # jak będzie czas to zmodyfikować, żeby zera nie dodawał
        row_winner = None
        pair_counter = 0

        for each_row in self.game_board:
            predecessor = each_row[0]
            for each_position in range(1, len(each_row)):
                if predecessor != each_row[each_position]:
                    break
                pair_counter += 1
                predecessor = each_row[each_position]

            if pair_counter == len(each_row) - 1:
                row_winner = predecessor
                return row_winner
            pair_counter = 0
        return row_winner

    def vertical_win(self):
        column_winner = None
        pair_counter = 0

        for each_column in range(len(self.game_board[0])):
            predecessor = self.game_board[0][each_column]
            for each_position in range(1, len(self.game_board)):
                if predecessor != self.game_board[each_position][each_column]:
                    break
                pair_counter += 1
                predecessor = self.game_board[each_position][each_column]

            if pair_counter == len(self.game_board) - 1:
                column_winner = predecessor
                return column_winner
            pair_counter = 0
        return column_winner

    def first_diagonal_win(self):
        diagonal_winner = None
        pair_counter = 0
        predecessor = self.game_board[0][0]

        for position in range(1, self.size_row):
            if predecessor != self.game_board[position][position]:
                return diagonal_winner
            pair_counter += 1
            predecessor = self.game_board[position][position]

        if pair_counter == len(self.game_board) - 1:
            diagonal_winner = predecessor
        return diagonal_winner

    def second_diagonal_win(self):
        diagonal_winner = None
        pair_counter = 0
        predecessor = self.game_board[0][self.size_row - 1]

        for position in range(1, self.size_row):
            if predecessor != self.game_board[position][self.size_row - 1 - position]:
                return diagonal_winner
            pair_counter += 1
            predecessor = self.game_board[position][self.size_row - 1 - position]

        if pair_counter == len(self.game_board) - 1:
            diagonal_winner = predecessor
        return diagonal_winner

    def graphic_rep(self):
        """
        Shows graphic representation of Tic Tac Toe game.
        Draws graph (with correct "location" of legends)
        for game boards with sizes between 1-99.
        :return:
        """
        board = self.game_board
        # size = np.shape(board.game_board())[0]
        for numbers in range(self.size_row+1):
            if numbers == 0:
                numbers = '  '
                print(numbers, end=' ')
            else:
                if numbers < 10:
                    print(f' {numbers}', end=' ')
                else:
                    print(f'{numbers}', end=' ')
        print()
        for numbers in range(self.size_row):
            print(f'{(numbers + 1):2}', end=" ")
            for element in board[numbers, :-1]:
                if element == 0:
                    print(" .", end=" ")
                elif element == 1:
                    print(" X", end=" ")
                elif element == 2:
                    print(" O", end=" ")
            last_element = board[numbers, -1]
            if last_element == 0:
                print(" .")
            elif last_element == 1:
                print(" X")
            elif last_element == 2:
                print(" O")



### Algorytm Minimax

#### Zmienne globalne
- α oraz β

In [ ]:
alfa = float('-inf')
beta = float('inf')

####

In [ ]:
class MinMaxAlgorithm:
    def __init__(self, game: callable):
        self.game = game

    def minimax(self, current_node: TicTacToe, depth: int, alfa: float, beta: float, is_maximizing_player: bool):  # current node musi być tablicą a nie nodem
        if current_node.is_terminal() or depth == 0:
            return current_node.heuristic_function()  # co powinno zwracać??

        if is_maximizing_player:
            children_nodes = current_node.evaluate_children(is_maximizing_player)
            for each_child in children_nodes:
                new_alfa = max(alfa, self.minimax(each_child, depth - 1, alfa, beta, not is_maximizing_player))
                if new_alfa >= beta:
                    return beta
            return alfa
        else:
            children_nodes = current_node.evaluate_children(not is_maximizing_player)
            for each_child in children_nodes:
                new_beta = min(beta, self.minimax(each_child, depth - 1, alfa, beta, is_maximizing_player))
                if alfa >= new_beta:
                    return new_beta
            return beta

In [ ]:
class Game:
    def __init__(self, tictactoe_board: TicTacToe):
        self.board = tictactoe_board

    def initialize_game(self):
        pass

    def round_game(self):
        pass

    def show_results_after_round(self):
        pass
